# Day 4 "Giant Squid"

## Part 1

### Problem

You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

        7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

        22 13 17 11  0
         8  2 23  4 24
        21  9 14 16  7
         6 10  3 18  5
         1 12 20 15 19

         3 15  0  2 22
         9 18 13 17  5
        19  8  7 25 23
        20 11 10 24  4
        14 21 16 12  6

        14 21 17 24  4
        10 16 15  9 19
        18  8 23 26 20
        22 11 13  6  5
         2  0 12  3  7

After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):

        22 13 17  *  0         3 15  0  2 22        14 21 17 24  *
         8  2 23  * 24         * 18 13 17  *        10 16 15  * 19
        21  * 14 16  *        19  8  * 25 23        18  8 23 26 20
         6 10  3 18  *        20  * 10 24  *        22  * 13  6  *
         1 12 20 15 19        14 21 16 12  6         2  0 12  3  *

After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:

        22 13  *  *  *         3 15  *  * 22         *  *  * 24  *
         8  *  *  * 24         * 18 13  *  *        10 16 15  * 19
         *  *  * 16  *        19  8  * 25  *        18  8  * 26 20
         6 10  3 18  *        20  * 10 24  *        22  * 13  6  *
         1 12 20 15 19         *  * 16 12  6         *  * 12  3  *

Finally, 24 is drawn:

        22 13  *  *  *         3 15  *  * 22         *  *  *  *  *
         8  *  *  * 24         * 18 13  *  *        10 16 15  * 19
         *  *  * 16  *        19  8  * 25  *        18  8  * 26 20
         6 10  3 18  *        20  * 10 24  *        22  * 13  6  *
         1 12 20 15 19         *  * 16 12  6         *  * 12  3  *

At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: 14 21 17 24 4).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

### Setup

For the board state, keeping it simple and stupid:

* The boards are kept as a flat array of strings
* Access using striding and offsets
* Convert to ints only for the board score.
* Used slots are marked with `*`.

We start with basic functions to parse, dump, and query the boards, given this data model.

In [1]:
from utils import *

_input = initDay("day4")

_sample = '''
    7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

    22 13 17 11  0
     8  2 23  4 24
    21  9 14 16  7
     6 10  3 18  5
     1 12 20 15 19

     3 15  0  2 22
     9 18 13 17  5
    19  8  7 25 23
    20 11 10 24  4
    14 21 16 12  6

    14 21 17 24  4
    10 16 15  9 19
    18  8 23 26 20
    22 11 13  6  5
     2  0 12  3  7
'''

def getCell(boards, b, x, y):
    return boards[b*25 + y*5 + x]

def dumpBoards(boards):
    bc = len(boards)//25
    for b in range(math.ceil(bc/3)):
        for y in range(5):
            for i in range(3):
                if b+i < bc:
                    for x in range(5):
                        cell = getCell(boards, b+i, x, y)
                        print(f'{cell:>2}', end=(' '))
                    print('   ', end='')
            print()
        print()

def parseItems(text):
    return [v for v in re.findall('\d+', text)] 

def parse(text):
    lines = [l for l in text.splitlines() if l]
    return parseItems(lines[0]), parseItems(' '.join(lines[1:]))

### Solution

Basic flow for each move:

* Replace slots matching the move with `*`.
* Check all boards for any fully-used rows or columns.
* If we want the first winning board, calculate its score and return it.
* Otherwise, remove the board and keep going until we're on the last board, then return its score instead.

In [2]:
def solve(text, takeFirst):

    offsets = list(itertools.chain(*[
        [[y*5+x for x in range(5)] for y in range(5)], # horiz
        [[y*5+x for y in range(5)] for x in range(5)]  # vert
    ]))

    moves, boards = parse(text)

    for move in moves:
        boards = ['*' if c == move else c for c in boards]
        remove = []

        for board in range(boardCount := len(boards)//25):
            for group in offsets:
                if sum([boards[board*25+o] == '*' for o in group]) == 5:
                    if takeFirst or boardCount == 1:
                        total = 0
                        for xy in [(x, y) for y in range(5) for x in range(5)]:
                            if (c := getCell(boards, board, xy[0], xy[1])) != '*':
                                total += int(c)
                        return total * int(move)
                    remove.append(board)

        for board in reversed(remove):
            boards = boards[:board*25]+boards[(board+1)*25:]

def solve1(text):
    return solve(text, True)

check(solve1(_sample), 4512)
check1(solve1(_input))

Part 1 Result: 69579


## Part 2

### Problem

On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

Figure out which board will win last. Once it wins, what would its final score be?

### Solution

Call generalized solution, telling it not to take the first winning board.

In [3]:
def solve2(text):
    return solve(text, False)

check(solve2(_sample), 1924)
check2(solve2(_input))

Part 2 Result: 14877
